Preliminary work:
    - count 0s, 0.1s, and missing values

In [62]:
import synapseclient
import pandas as pd
import numpy as np
import json

syn = synapseclient.Synapse()
syn.login(silent=True)

table_id = 'syn25575156' # old one: syn25905783
overall_scores = syn.tableQuery(str("select * from {0}".format(table_id))).asDataFrame() # convert to syn_query
overall_scores.columns

Index(['GeneName', 'ENSG', 'Overall', 'Overall_rank', 'GeneticsScore',
       'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore',
       'SM_Druggability_bucket', 'safety_bucket', 'feasibility_bucket',
       'ABability_bucket', 'new_modality_bucket', 'tissue_engagement_bucket',
       'Pharos_class', 'classification', 'safety_bucket_definition',
       'feasibility_bucket_definition', 'ABability_bucket_definition',
       'new_modality_bucket_definition', 'tissue_engagement_bucket_definition',
       'isScored_genetics', 'isScored_omics', 'isScored_lit',
       'isScored_neuropath'],
      dtype='object')

The pipeline needs the code below instead of the code block the notebook uses

In [63]:
# interesting_columns = ['overall', 'geneticsscore', 'omicsscore', 'literaturescore']
# overall_scores = overall_scores[interesting_columns]

In [64]:
overall_scores = overall_scores[['Overall', 'GeneticsScore', 'OmicsScore', 'LiteratureScore']]

In [65]:
neo_matrix = {}
for col in overall_scores.columns:
    neo_matrix[col] = {}

    try:
        neo_matrix[col]["distribution"] = list(pd.cut(overall_scores[col], bins=10, precision=3).value_counts())
    except TypeError:
        overall_scores[col] = pd.to_numeric(overall_scores[col], errors='coerce')
        neo_matrix[col]["distribution"] = list(pd.cut(overall_scores[col], bins=10, precision=3).value_counts())
        
    neo_matrix[col]["min"] = np.around(overall_scores[col].min(), 3)
    neo_matrix[col]["max"] = np.around(overall_scores[col].max(), 3)
    neo_matrix[col]["mean"] = np.around(overall_scores[col].mean(), 3)
    discard, neo_matrix[col]["bins"] = list(pd.cut(overall_scores[col], bins=10, precision=3, retbins=True))
    neo_matrix[col]["bins"] = np.around(neo_matrix[col]["bins"].tolist()[1:], 3)
    base = [0, *neo_matrix[col]["bins"][:-1]]
    neo_matrix[col]["bins"] = zip(base, neo_matrix[col]["bins"])
    neo_matrix[col]["bins"] = list(neo_matrix[col]["bins"])
    neo_matrix[col]["first_quartile"] = np.around(overall_scores[col].quantile(q=0.25, interpolation='midpoint'))
    neo_matrix[col]["third_quartile"] = np.around(overall_scores[col].quantile(q=0.75, interpolation='midpoint'))

In [66]:
for col in neo_matrix.keys():
    print(col)
    print(neo_matrix[col])

Overall
{'distribution': [5141, 4253, 4143, 3590, 3146, 1733, 975, 346, 41, 18], 'min': 0.0, 'max': 7.537, 'mean': 2.175, 'bins': [(0, 0.754), (0.754, 1.507), (1.507, 2.261), (2.261, 3.015), (3.015, 3.769), (3.769, 4.522), (4.522, 5.276), (5.276, 6.03), (6.03, 6.784), (6.784, 7.537)], 'first_quartile': 1.0, 'third_quartile': 3.0}
GeneticsScore
{'distribution': [4760, 4693, 4679, 2993, 2346, 2055, 1206, 570, 75, 9], 'min': 0.0, 'max': 3.0, 'mean': 1.171, 'bins': [(0, 0.3), (0.3, 0.6), (0.6, 0.9), (0.9, 1.2), (1.2, 1.5), (1.5, 1.8), (1.8, 2.1), (2.1, 2.4), (2.4, 2.7), (2.7, 3.0)], 'first_quartile': 1.0, 'third_quartile': 2.0}
OmicsScore
{'distribution': [11138, 2523, 2298, 1568, 1278, 1254, 1145, 873, 852, 457], 'min': 0.0, 'max': 2.0, 'mean': 0.661, 'bins': [(0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0), (1.0, 1.2), (1.2, 1.4), (1.4, 1.6), (1.6, 1.8), (1.8, 2.0)], 'first_quartile': 0.0, 'third_quartile': 1.0}
LiteratureScore
{'distribution': [16232, 1390, 1103, 1086, 730, 723

In [69]:
with open('../output/distribution_data.json', 'w') as outfile:
    json.dump(neo_matrix, outfile)